### Carga de los datos:

In [2]:
%pip install mysql.connector

%pip install xlrd

%pip install pandas

%pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 49.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247953 sha256=8bb304247742b8361c3ff42072add146960fe5ec42f33c6b99762fe4535a992e
  Stored in directory: /home/codespace/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql.connector
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 2.5 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import xlrd
import mysql.connector
import pandas as pd
import os

In [4]:
# Cargar variables de entorno
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

In [5]:
# Conectarse a la base de datos MySQL usando las variables de entorno creadas
conexion = mysql.connector.connect(
    host=os.getenv("EV_host"),
    user=os.getenv("EV_user"),
    password=os.getenv("EV_password"),
    database=os.getenv("EV_database"),
)

In [6]:
cursor = conexion.cursor()                      # Instalamos un cursor para ejecutar consultas SQL

In [7]:
nombre_base_datos = "sandbox"                   # Definimos como variables la bbdd y las tablas
nombre_tabla_ventas = "raw_ventas"
nombre_tabla_calendario = "raw_calendario"
nombre_tabla_pedidos = "raw_pedidos"

cursor.execute(f"USE {nombre_base_datos}")      # Para usar la bbdd sandbox

In [ ]:
# Borramos la tabla de ventas que haya si la hay, y creamos la tabla de ventas que usaremos en este proyecto, seleccionando el tipo de dato para cada campo. Hacemos lo mismo en las siguientes dos celdas con las otras tablas
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_ventas}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_ventas} ("
               "FAMILIA VARCHAR(255),"
               "Tipo VARCHAR(255),"
               "FechaVenta DATE,"
               "HoraVenta INT,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [ ]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_calendario}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_calendario} ("
               "Fecha DATE,"
               "Festivo VARCHAR(255)"
               ")")

In [ ]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_pedidos}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_pedidos} ("
               "Tipo VARCHAR(255),"
               "Fecha DATE,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [8]:
# Para trabajar con archivos .xlsx en el entorno
%pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Cargar el archivo Excel
nombre_archivo_ventas = "data/ArticulosPanaderia.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_ventas)

# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    familia = row['FAMILIA']
    tipo = row['Tipo']
    fecha_venta = row['FechaVenta'].strftime('%Y-%m-%d')
    hora_venta = row['HoraVenta']
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append((familia, tipo, fecha_venta, hora_venta, articulo, cantidad, precio, importe))

# Seleccionar la base de datos
cursor.execute("USE sandbox")

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_ventas} (FAMILIA, Tipo, FechaVenta, HoraVenta, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [ ]:
# Cargar el archivo Excel
nombre_archivo_calendario = "data/Calendario.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_calendario)

# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    fecha = row['Fecha'].strftime('%Y-%m-%d')
    festivo = row['Festivo']
    registros.append((fecha,festivo))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_calendario} (Fecha, Festivo) " \
           "VALUES (%s, %s)"

# Seleccionar la base de datos          #
cursor.execute("USE sandbox")

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()


In [ ]:
# Cargar el archivo Excel
nombre_archivo_pedidos = "data/CantidadPedida.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_pedidos)

# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    tipo = row['Tipo']
    fecha_venta = row['Fecha'].strftime('%Y-%m-%d')
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append(( tipo, fecha_venta, articulo, cantidad, precio, importe))

# Seleccionar la base de datos
cursor.execute("USE sandbox")

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_pedidos} (Tipo, Fecha, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s)"

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [9]:
#Guardamos en una variable una serie de consultas para ejecutar más adelante

consulta_proceso="""


/*Ordenamos los productos por familia*/
drop table if exists sandbox.articulos_top;
create table sandbox.articulos_top as (

	select
	Articulo,
	FAMILIA,
	sum(importe) as importe_total,
	ROW_NUMBER () OVER(partition by FAMILIA order by sum(importe) desc) as orden
	from sandbox.raw_ventas
	where FechaVenta >= '2021-05-01'
	group by 1,2

);


/*Creamos un calendario general del 1/1/2017 al 31/12/2023*/
drop table if exists sandbox.calendario_dias;
create table sandbox.calendario_dias as (

with recursive cte_calendario as (
	select date('2017-01-01') as calendar_date -- fecha de inicio
	union all
	select date_add(calendar_date, interval 1 day) as calendar_date from cte_calendario
	where date_add(calendar_date, interval 1 day) <= date('2023-12-31') -- condicion de fin, fecha final
)

select
calendar_date as fecha,
year(calendar_date) as fx_anno,
month(calendar_date) as fx_mes,
day(calendar_date) as fx_day,
date_format(calendar_date, '%Y%m') as fx_anno_mes,
date_format(calendar_Date,'%x-%v') as semana -- formato
from cte_calendario

);

/*Añadimos al calendario los festivos*/
drop table if exists sandbox.calendario_completo;
create table sandbox.calendario_completo as (
select
base.*,
festivos.festivo
from sandbox.calendario_dias base
left join
	(select
	a.*,
	row_number() over(partition by a.fecha order by a.festivo) as orden
	from sandbox.raw_calendario a
	) festivos
on base.fecha=festivos.fecha
and festivos.orden=1
);

/*Agregamos las ventas por día (quitamos el detalle por hora)*/

/*Añadimos además el orden del producto y si está en el rango de fecha de estudio*/
drop table if exists sandbox.ventas_diarias;
create table sandbox.ventas_diarias as (
select
base.familia,
base.tipo,
base.fechaVenta,
calendario.festivo,
base.articulo,
sum(base.precio*base.cantidad)/sum(base.cantidad) as precio, /* Se calcula como el precio promedio del día, ya que hay cambios de precio a lo largo del día*/
articulos.orden as orden_articulo_familia,
case when base.fechaVenta >=date('2021-05-01') then 'S' else 'N' end as in_fecha_estudio,
sum(base.cantidad) as cantidad,
sum(base.importe) as importe

from sandbox.raw_ventas base

inner join sandbox.calendario_completo calendario
on base.FechaVenta=calendario.fecha

inner join sandbox.articulos_top articulos
on base.familia=articulos.familia
and base.articulo=articulos.articulo


group by 1,2,3,4,5,7,8);


/*Creamos una vista final con el filtrado para el estudio*/
drop view if exists sandbox.ventas_diarias_estudio_completo;
create view sandbox.ventas_diarias_estudio_completo as
(select *
from sandbox.ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and orden_articulo_familia<=5);


/*Creamos una vista extra con el filtrado para el estudio que no tenga días de mayo 23 (que habrá que predecir)*/
drop view if exists sandbox.ventas_diarias_estudio;
create view sandbox.ventas_diarias_estudio as
(select *
from sandbox.ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and fechaventa < date('2023-05-01')
and orden_articulo_familia<=5);
"""

In [10]:
# Separar las consultas en una lista
consultas_lista = consulta_proceso.split(";")

# Eliminar espacios en blanco y elementos vacíos
consultas_lista = [consulta.strip() for consulta in consultas_lista if consulta.strip()]

# Ejecutar las consultas
for consulta in consultas_lista:
    cursor.execute(consulta)

# Confirmar los cambios
conexion.commit()

### Obtención de los datos:

In [27]:
!pip install SQLAlchemy==1.4.48
!pip install pymysql

import sqlalchemy as sa
import pymysql
import sqlite3

In [29]:
class DatabaseConnection:
    # Información necesaria para establecer la conexión a la BBDD:
    def __init__(self, password,
                 username="usuario1",
                 host="database-1.cxlpff3hacbu.eu-west-3.rds.amazonaws.com",
                 port=3306,
                 database="classicmodels"):

        self.__password = password
        self.connection = pymysql.connect(
            host=host,
            user=username,
            password=password,
            db=database,
            port=port,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )

    def query_to_df(self, query):
        try:
            with self.connection.cursor() as cursor:
                cursor.execute(query)
                result = cursor.fetchall()
                df = pd.DataFrame(result)
            return df
        except Exception as e:
            print("Error al ejecutar la consulta:", e)
        finally:
            self.connection.close()

# Query a consultar. Cambio de nombre de fechaVenta.
# IMP: se omiten las columnas irrelevantes -> 'in_fecha_estudio' y 'tipo'
query = """
SELECT *
FROM ventas_diarias_estudio
WHERE ARTICULO = '3960'
"""

# Instaciar la clase con la contraseña de la BBDD
database = DatabaseConnection("C0d35p4ce.")

# Consultar la query anterior y devolverla como un dataframe de Pandas
ventas = database.query_to_df(query)



ventas


,familia,tipo,fechaVenta,festivo,articulo,precio,orden_articulo_familia,in_fecha_estudio,cantidad,importe
0,BOLLERIA,VENTA,2021-05-18,None,3960,2.318,1,S,216.0,500.688008
1,BOLLERIA,VENTA,2022-05-18,None,3960,2.591,1,S,150.0,388.650003
2,BOLLERIA,VENTA,2022-09-13,None,3960,2.591,1,S,192.0,497.472011
3,BOLLERIA,VENTA,2022-12-12,None,3960,2.727,1,S,129.0,351.782990
4,BOLLERIA,VENTA,2021-07-29,None,3960,2.318,1,S,279.0,646.722011
...,...,...,...,...,...,...,...,...,...,...
725,BOLLERIA,VENTA,2022-02-26,None,3960,2.591,1,S,387.0,1002.717010
726,BOLLERIA,VENTA,2022-04-13,Miércoles Santo,3960,2.591,1,S,282.0,730.662001
727,BOLLERIA,VENTA,2022-07-09,None,3960,2.591,1,S,195.0,505.245007
728,BOLLERIA,VENTA,2022-07-21,None,3960,2.591,1,S,126.0,326.466006


### EDA

In [33]:
!pip install meteostat
!pip install statsmodels
!pip install missingno

In [34]:
import numpy as np

import datetime

from meteostat import Point, Daily
import sqlalchemy as sa

from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

import plotly.express as px
import matplotlib.pyplot as plt
import missingno as msno

In [37]:
class DatabaseConnection:
    # Información necesaria para establecer la conexión a la BBDD:
    def __init__(self, password,
                 username="usuario1",
                 host="database-1.cxlpff3hacbu.eu-west-3.rds.amazonaws.com",
                 port=3306,
                 database="classicmodels"):

        self.__password = password
        self.connection = pymysql.connect(
            host=host,
            user=username,
            password=password,
            db=database,
            port=port,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )

    def query_to_df(self, query):
        try:
            with self.connection.cursor() as cursor:
                cursor.execute(query)
                result = cursor.fetchall()
                df = pd.DataFrame(result)
            return df
        except Exception as e:
            print("Error al ejecutar la consulta:", e)
        finally:
            self.connection.close()

# Query a consultar. Cambio de nombre de fechaVenta.
# IMP: se omiten las columnas irrelevantes -> 'in_fecha_estudio' y 'tipo'
query_daily_and_top = """
SELECT familia,
articulo,
fechaVenta AS fecha_venta,
festivo,
precio,
cantidad,
importe,
orden_articulo_familia

FROM ventas_diarias_estudio
"""

# Instaciar la clase con la contraseña de la BBDD
database = DatabaseConnection("C0d35p4ce.")

# Consultar la query anterior y devolverla como un dataframe de Pandas
ventas = database.query_to_df(query_daily_and_top)

# Convertir 'fecha_venta' a un formato de fecha y 'articulo' a entero
ventas = (
           ventas.assign(fecha_venta = pd.to_datetime(ventas["fecha_venta"]))
                 .astype({"articulo": int})
        )

ventas






,familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia
0,BOLLERIA,3960,2021-05-18,None,2.318000,216.000000,500.688008,1
1,BOLLERIA,6286,2021-08-29,None,3.136000,99.000000,310.464005,4
2,BOLLERIA,3880,2021-09-21,None,3.000000,102.000000,306.000000,3
3,BOLLERIA,5803,2021-10-21,None,3.000000,42.000000,126.000000,5
4,BOLLERIA,3880,2021-12-07,None,3.000000,141.000000,423.000000,3
...,...,...,...,...,...,...,...,...
10317,PASTELERIA,6451,2023-02-07,None,43.500000,3.000000,130.500000,4
10318,PASTELERIA,6523,2023-04-01,None,40.772999,18.000000,733.914001,3
10319,PASTELERIA,6451,2023-04-02,Domingo de Ramos,43.500000,18.000000,783.000000,4
10320,PASTELERIA,5404,2023-04-15,None,43.500000,52.199999,2270.700058,1


## Consulta a [meteostat API](https://dev.meteostat.net/python/daily.html)

Variables que se obtienen:

- **tavg** -> The average air temperature in °C
- **tmin** -> The minimum air temperature in °C
- **tmax** -> The maximum air temperature in °C
- **prcp** -> The daily precipitation total in mm
- **wdir** -> The average wind direction in degrees (°)
- **wspd** -> The average wind speed in km/h
- **pres** -> The average sea-level air pressure in hPa

In [38]:
class DailyWeatherData:

    # Localización de la tienda de la Panadería Salvador Echeverría
    ECHEVERRIA_SHOP = Point(36.709758, -4.462520)

    # Columnas identificadas como importantes
    IMP_COLUMNS = ['tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'pres']

    # Definición del horizonte temporal de la consulta a la API
    def __init__(self, start, end = datetime.datetime.now()):
        self.start = start
        self.end = end

    # Se establece que la consulta será diaria, se seleccionan todas las columnas imps y se añade un suff.
    def get_weather_data(self):
        daily_data = Daily(self.ECHEVERRIA_SHOP, self.start, self.end)
        data = daily_data.fetch()
        return data[self.IMP_COLUMNS].add_suffix('_w').reset_index()

In [39]:
# Fecha en la que comienza el conjunto de datos de ventas
# Se puede definir también en este formato -> Ej: datetime.datetime(2017, 1, 1)
start = ventas["fecha_venta"].min()

# Fecha en la que termina el conjunto de datos de ventas
# Se puede definir también en este formato -> Ej: datetime.datetime(2017, 1, 1)
end = ventas["fecha_venta"].max()

# Instancia la clase con el horizonte temporal a consultar
weather_query = DailyWeatherData(start, end)

# Obtener la info del tiempo del horizonte temporal consultado
weather = weather_query.get_weather_data()

weather

,time,tavg_w,tmin_w,tmax_w,prcp_w,wdir_w,wspd_w,pres_w
0,2021-05-01,17.2,11.9,22.8,0.0,282.0,14.2,1014.0
1,2021-05-02,16.3,10.8,23.2,0.0,233.0,15.7,1015.9
2,2021-05-03,15.3,12.7,20.1,6.5,272.0,9.7,1017.7
3,2021-05-04,16.5,12.0,20.9,0.0,206.0,12.1,1018.0
4,2021-05-05,16.7,12.3,21.7,0.0,132.0,13.8,1016.5
...,...,...,...,...,...,...,...,...
725,2023-04-26,20.3,15.5,26.0,0.0,93.0,10.7,1014.0
726,2023-04-27,19.1,15.0,24.5,0.0,122.0,9.9,1015.1
727,2023-04-28,20.8,16.6,26.7,0.0,144.0,14.3,1015.2
728,2023-04-29,25.6,18.7,34.3,0.0,289.0,16.9,1015.4


In [40]:
# Solo dos días tienen valores nulos en 3 de 7 columnas
weather.isnull().sum()

time      0
tavg_w    0
tmin_w    0
tmax_w    0
prcp_w    0
wdir_w    2
wspd_w    2
pres_w    2
dtype: int64

## Join -> Datos de ventas y del tiempo

In [41]:
# Join del conjunto de datos de ventas y del tiempo por medio de la columna de fecha
df = pd.merge(ventas, weather,
              left_on=["fecha_venta"], right_on=["time"],
              how="inner").drop("time", axis=1)

# Ordenar el dataframe
df.sort_values(["articulo", "fecha_venta"], inplace = True)

df.head(5)

,familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia,tavg_w,tmin_w,tmax_w,prcp_w,wdir_w,wspd_w,pres_w
8460,PANADERIA,417,2021-05-01,None,4.038,72.0,290.736004,5,17.2,11.9,22.8,0.0,282.0,14.2,1014.0
436,PANADERIA,417,2021-05-02,Día de la Madre,4.038,81.0,327.077995,5,16.3,10.8,23.2,0.0,233.0,15.7,1015.9
1758,PANADERIA,417,2021-05-03,Día de la Cruz,4.038,90.0,363.419998,5,15.3,12.7,20.1,6.5,272.0,9.7,1017.7
5867,PANADERIA,417,2021-05-04,None,4.038,84.0,339.192001,5,16.5,12.0,20.9,0.0,206.0,12.1,1018.0
10022,PANADERIA,417,2021-05-05,None,4.038,84.0,339.192003,5,16.7,12.3,21.7,0.0,132.0,13.8,1016.5


## PROCESO DE ANÁLISIS DE DATOS EXPLORATORIO (EDA)